<a href="https://colab.research.google.com/github/suriarasai/BEAD2025/blob/main/colab/09SparkDataFrameSQLDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PYSPARK SQL and DataFrame
The goal of this tutorial is to train you on how to create data access classes using the Spark to retrieve data from a NoSQL data source.
##Rebu Taxi Case Study:
The Rebu Taxi Case Study introduced in the IADD course is used here for performing some Spark Query and Analytics. For this workshop four data stores are provided viz., Driver Data, Passenger Data, Taxi Data and Trip Data. The next section lists the filenames for these, three of which are in CSV format and the fourth in JSON format. For further consolidation of the learning, a set of exercises are provided based on Rebu Taxi Booking Case study, which the student should complete to attain the required learning outcome.

For demonstration purposes and workshop practice we provide the following files containing the data:


1. Customer.CSV
2. CustomerNoHdr.CSV
3. Country.JSON (for practicing alternate data format i.e. JSON files)
4. BEAD_Rebu_Drivers.CSV
5. BEAD_Rebu_Passengers.CSV
6. BEAD_Rebu_TaxiCabs.JSON
7. BEAD_Rebu_TripData.CSV

Please upload each of the above files to your Google Drive, placing them in the designated 'data' subfolder.


## PySpark Install

The first step involves installing pyspark.

In [1]:
import os

# 1. Install OpenJDK 21 (if not already done in a previous cell)
!apt-get update -qq
!apt-get install -qq openjdk-21-jdk-headless

# 2. Verify where it landed (if needed)
!ls /usr/lib/jvm | grep 21

# 3. Point to JDK 21
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

# 4. Install PySpark via pip (make sure this happens AFTER setting JAVA_HOME)
!pip install pyspark --quiet

# 5. Import and start Spark
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
      .master("local[*]")
      .appName("Spark on Java21")
      .getOrCreate()
)



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package openjdk-21-jre-headless:amd64.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../openjdk-21-jre-headless_21.0.7+6~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-21-jre-headless:amd64 (21.0.7+6~us1-0ubuntu1~22.04) ...
Selecting previously unselected package openjdk-21-jdk-headless:amd64.
Preparing to unpack .../openjdk-21-jdk-headless_21.0.7+6~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-21-jdk-headless:amd64 (21.0.7+6~us1-0ubuntu1~22.04) ...
Setting up openjdk-21-jre-headless:amd64 (21.0.7+6~us1-0ubuntu1~22.04) ...
update-alternatives: using /usr/lib/jvm/java-21-openjdk-amd64/bin/java to provide /usr/bin/java (java) in auto mode
update-alternatives: using /usr/lib/jvm/java-21-openjdk-amd64/bin/jpackage to

## Mounting Google Drive
Connect to Google Drive as the next step.

In [2]:
# to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
from google.colab import drive
# to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Spark Query Examples

In this section we systematically walk through various queries starting from the simple ones and then build on to more complex queries. Walking through these self-guiding examples facilitates the students to gain comfort in handling various types of queries.

Python Spark provides the option of embedding standard SQL queries. While we introduce it later for completeness, we will confine to functional programming approach in this workshop which is the more acceptable way of coding in the modern day systems.

To get participants to speed, we confine to the CSV data deployed in a local folder, and walk the Scala Queries. It should be noted, that the queries themselves are IDENTICAL and INDEPENDENT of the data source and hence will work irrespective of whether the source is csv, json or rdbms database.

###Example 1: Retrieving all data from the Customers Dataset

In [10]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.master("local").appName("Customer").getOrCreate()
inputFilePath = "/content/drive/My Drive/data/Customer.csv"
df_cust = ( spark.read
.option("header", "true")
.option("inferSchema", "true")
.csv(inputFilePath) )
df_cust.show()

+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|        CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      1000|        Lou Anna Tan|             A| 29|     F|       4.14|Blk 26, Telok Bla...|Frankfurt|        GER|          Ms|    2732287|
|      1001|      Wong Sook Huey|             A| 37|     F|       67.1|Blk 1007 Teresa V...|Singapore|        SIN|          Ms|    2740975|
|      1002|       Ng Choon Seng|             C| 23|     M|      63.18|Blk 63 Bishan St ...|  Toronto|        CAN|          Mr|    2580742|
|      1003|      Chew Teck Kuan|             C| 63|     M|      64.49|Blk 109 Bedok Nor...|Singapore|        SIN|          Mr|    4434675|
|      1111|        

To see full set column width and more rows of data, you may use the following option of the show method. (i.e., up to 200 records are shown and column truncation set to false)

In [11]:
df_cust.show(200, False)

+----------+----------------------+--------------+---+------+-----------+-------------------------------------------------------------+---------+-----------+------------+-----------+
|CustomerID|CustomerName          |MemberCategory|Age|Gender|AmountSpent|Address                                                      |City     |CountryCode|ContactTitle|PhoneNumber|
+----------+----------------------+--------------+---+------+-----------+-------------------------------------------------------------+---------+-----------+------------+-----------+
|1000      |Lou Anna Tan          |A             |29 |F     |4.14       |Blk 26, Telok Blangah Crescent #22-87, Singapore 0409        |Frankfurt|GER        |Ms          |2732287    |
|1001      |Wong Sook Huey        |A             |37 |F     |67.1       |Blk 1007 Teresa Ville Lower Delta Road #06-02, Singapore 0410|Singapore|SIN        |Ms          |2740975    |
|1002      |Ng Choon Seng         |C             |23 |M     |63.18      |Blk 63 Bisha

###Exploring Schema:
To explore the implicit schema that has been generated based on the header row and interpretation of the data type in the subsequent data row you may use the following statement:

In [12]:
df_cust.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- MemberCategory: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- AmountSpent: double (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- ContactTitle: string (nullable = true)
 |-- PhoneNumber: integer (nullable = true)



###Setting Schema:
If you do not wish to use the implicit schema or if there are no header row in the file, then you can set your own definitions for column headers and the data types. There are a few ways of doing that, which you may explore from the api reference documentation.
One of the ways of doing is to define a schema and use that schema while reading the csv, as shown in codes below:

In [13]:
inputFilePath = "/content/drive/My Drive/data/CustomerNoHdr.csv"
custschema = StructType([ StructField("Customerid", IntegerType(), True),
	StructField("CustName", StringType(), True),
	StructField("MemCat", StringType(), True),
	StructField("Age", IntegerType(), True),
	StructField("Gender", StringType(), True),
	StructField("AmtSpent", DoubleType(), True),
	StructField("Address", StringType(), True),
	StructField("City", StringType(), True),
	StructField("CountryID", StringType(), True),
	StructField("Title", StringType(), True),
	StructField("PhoneNo", StringType(), True) ])
df = spark.read .schema(schema=custschema).csv(inputFilePath)
df.printSchema()
df.show()

root
 |-- Customerid: integer (nullable = true)
 |-- CustName: string (nullable = true)
 |-- MemCat: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- AmtSpent: double (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- CountryID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- PhoneNo: string (nullable = true)

+----------+--------------------+------+---+------+--------+--------------------+---------+---------+-----+-------+
|Customerid|            CustName|MemCat|Age|Gender|AmtSpent|             Address|     City|CountryID|Title|PhoneNo|
+----------+--------------------+------+---+------+--------+--------------------+---------+---------+-----+-------+
|      1000|        Lou Anna Tan|     A| 29|     F|    4.14|Blk 26, Telok Bla...|Frankfurt|      GER|   Ms|2732287|
|      1001|      Wong Sook Huey|     A| 37|     F|    67.1|Blk 1007 Teresa V...|Singapore|      SIN|  

###Example 2: Returning Selected Fields

Show only CustomerID, CustomerName and Age

Note that when we return a dataset (customers in the above case), we get all columns.

If we wish to return only selected fields, e.g, if you wish to only return CustomerID, CustomerName and Age and skip the other fields then you can specify the selected fields.

In [16]:
df_cust.select(df_cust["CustomerID"], df_cust["CustomerName"], df_cust["Age"]).show()

+----------+--------------------+---+
|CustomerID|        CustomerName|Age|
+----------+--------------------+---+
|      1000|        Lou Anna Tan| 29|
|      1001|      Wong Sook Huey| 37|
|      1002|       Ng Choon Seng| 23|
|      1003|      Chew Teck Kuan| 63|
|      1111|           Steven Ou| 61|
|      1634|  Sridharan Jayanthi| 55|
|      1681|         Terence Lim| 30|
|      1810|         Vanessa Ong| 32|
|      1811|       Koh Ting Ting| 57|
|      1818|    Chionh Choon Lee| 57|
|      2131|                 Jon| 64|
|      2233|       Too Siew Hong| 35|
|      2270|   Chao Tah Jin Alex| 22|
|      2323|        Richard Kwan| 26|
|      2345| Ng Teck Kie Anthony| 56|
|      2626|          Steven Teo| 56|
|      2669|   Boh Lee Ming Lynn| 23|
|      2688|Kathleen Loh Swat...| 38|
|      2741|        Goh Chee Eng| 45|
|      2820|    Ng Wee Hock John| 56|
+----------+--------------------+---+
only showing top 20 rows



OR

In [19]:
df_cust.select("CustomerID", "CustomerName", "Age").show()

+----------+--------------------+---+
|CustomerID|        CustomerName|Age|
+----------+--------------------+---+
|      1000|        Lou Anna Tan| 29|
|      1001|      Wong Sook Huey| 37|
|      1002|       Ng Choon Seng| 23|
|      1003|      Chew Teck Kuan| 63|
|      1111|           Steven Ou| 61|
|      1634|  Sridharan Jayanthi| 55|
|      1681|         Terence Lim| 30|
|      1810|         Vanessa Ong| 32|
|      1811|       Koh Ting Ting| 57|
|      1818|    Chionh Choon Lee| 57|
|      2131|                 Jon| 64|
|      2233|       Too Siew Hong| 35|
|      2270|   Chao Tah Jin Alex| 22|
|      2323|        Richard Kwan| 26|
|      2345| Ng Teck Kie Anthony| 56|
|      2626|          Steven Teo| 56|
|      2669|   Boh Lee Ming Lynn| 23|
|      2688|Kathleen Loh Swat...| 38|
|      2741|        Goh Chee Eng| 45|
|      2820|    Ng Wee Hock John| 56|
+----------+--------------------+---+
only showing top 20 rows



OR

In [ ]:
df_cust.select(col("CustomerID"), col("CustomerName"), col("Age")).show()

##SPARK QUERY LANGUAGE FUNCTIONS
###Example 3: Retrieving data from the Customers Dataset & displaying sorted on Customer Name


In [20]:
df_cust.orderBy("CustomerName").show()

+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|        CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      4567|         Abdul Zaidi|             A| 29|     M|      31.65|Blk 24, Telok Bla...|Singapore|        SIN|          Mr|    2709466|
|      5489|        Ang Kim Beng|             A| 45|     M|      32.09|Blk 208 Bedok Nor...|Singapore|        SIN|          Mr|    4486578|
|      2669|   Boh Lee Ming Lynn|             C| 23|     F|      41.99|Blk 671, Woodland...|Singapore|        SIN|          Ms|    2234567|
|      8080|      Chan Chin Fung|             B| 56|     M|      24.95|6 Dover Rise, #17...|Singapore|        SIN|          Mr|    8738529|
|      2270|   Chao 

Getting sorted by member category first then on Customer Name:

In [22]:
df_cust.orderBy("MemberCategory", "CustomerName").show()


+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|        CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      4567|         Abdul Zaidi|             A| 29|     M|      31.65|Blk 24, Telok Bla...|Singapore|        SIN|          Mr|    2709466|
|      5489|        Ang Kim Beng|             A| 45|     M|      32.09|Blk 208 Bedok Nor...|Singapore|        SIN|          Mr|    4486578|
|      2983|          Cheryl Tan|             A| 37|     F|      13.99|Blk 84, Telok Bla...|Singapore|        SIN|          Ms|    2789967|
|      1818|    Chionh Choon Lee|             A| 57|     M|       7.13|Blk 89, Zion Road...|Singapore|        SIN|          Mr|    7333100|
|      5108|        

###Example 4: Selective retrieval
1. Obtaining only those Customers whose MemberCategory is ‘A’
      
      Use the filter function as below

In [23]:
df_cust.filter(df_cust["MemberCategory"] == "A").show()

+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|        CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      1000|        Lou Anna Tan|             A| 29|     F|       4.14|Blk 26, Telok Bla...|Frankfurt|        GER|          Ms|    2732287|
|      1001|      Wong Sook Huey|             A| 37|     F|       67.1|Blk 1007 Teresa V...|Singapore|        SIN|          Ms|    2740975|
|      1634|  Sridharan Jayanthi|             A| 55|     F|      61.51|Blk 232, Jurong E...|Singapore|        SIN|          Ms|    6658037|
|      1818|    Chionh Choon Lee|             A| 57|     M|       7.13|Blk 89, Zion Road...|Singapore|        SIN|          Mr|    7333100|
|      2131|        

2.  Obtaining only those Customers whose MemberCategory is ‘A’ & their name starts with ‘T’

In [24]:
df_cust.filter((df_cust["MemberCategory"] == "A") & (df_cust["CustomerName"].startswith("T"))).show()

+----------+---------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|   CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+---------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      3845|Tan Choon Heong|             A| 21|     M|      10.82|Blk 303, Shunfu R...|Singapore|        SIN|          Mr|    3580709|
|      5968|    Tan Wei Wei|             A| 48|     F|       71.6|3, Kismis Place, ...|Singapore|        SIN|          Ms|    4674463|
+----------+---------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+



###Example 5: Combining functions

In this example we will show the combination of filtering and sorting.

Retrieve all Customers who belongs to MemberCategory A and present in ascending order of names.

You will need to use multiple functions in succession as below.


In [25]:
df_cust.filter(df_cust["MemberCategory"] == "A").orderBy("CustomerName").show()

+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|CustomerID|        CustomerName|MemberCategory|Age|Gender|AmountSpent|             Address|     City|CountryCode|ContactTitle|PhoneNumber|
+----------+--------------------+--------------+---+------+-----------+--------------------+---------+-----------+------------+-----------+
|      4567|         Abdul Zaidi|             A| 29|     M|      31.65|Blk 24, Telok Bla...|Singapore|        SIN|          Mr|    2709466|
|      5489|        Ang Kim Beng|             A| 45|     M|      32.09|Blk 208 Bedok Nor...|Singapore|        SIN|          Mr|    4486578|
|      2983|          Cheryl Tan|             A| 37|     F|      13.99|Blk 84, Telok Bla...|Singapore|        SIN|          Ms|    2789967|
|      1818|    Chionh Choon Lee|             A| 57|     M|       7.13|Blk 89, Zion Road...|Singapore|        SIN|          Mr|    7333100|
|      5108|        

##AGGREGATION AND SIMPLE STATISTICAL QUERY EXAMPLES
###Example 6: Getting a count
Obtaining the number of A category members.

In [26]:
df_cust.filter(df_cust["MemberCategory"] == "A").count()

26

###Example 7: Getting the Sum of a field
Obtaining the Total Amount earned (i.e., sum of AmountSpent by all customers).


In [27]:
df_cust.agg(sum("AmountSpent")).show()

+-----------------+
| sum(AmountSpent)|
+-----------------+
|2486.309999999999|
+-----------------+



###Example 8: Getting the Sum of a field with condition on the rows to use
Obtaining the Total Amount earned from A category members (i.e., sum of AmountSpent by all customers whose MemberCategory is A).

In [28]:
df_cust.filter(df_cust["MemberCategory"] == "A").agg(sum("AmountSpent")).show()

+----------------+
|sum(AmountSpent)|
+----------------+
|         1194.66|
+----------------+



###Example 9: Getting the Average of a field
Obtaining the Average Age of customers

In [29]:
df_cust.agg(avg("Age")).show()

+--------+
|avg(Age)|
+--------+
|   41.82|
+--------+



##COMPLEX & FURTHER STASTICAL QUERIES
The example on Group by where sub-totals are obtained may be a good example to mix both.
###Example 10: Grouping & Subtotals based on single parameter
Obtaining Total Amount Spent for each all customers in each Member Category

In [30]:
df_cust.groupBy("MemberCategory").agg(sum("AmountSpent")).show()

+--------------+------------------+
|MemberCategory|  sum(AmountSpent)|
+--------------+------------------+
|             B|500.40999999999997|
|             C|            791.24|
|             A|           1194.66|
+--------------+------------------+



###Example 11: Grouping & Subtotals based on multiple parameters
Obtaining Total Amount Spent by customers based on their Member Category & Gender

In [32]:
df_cust.groupBy("MemberCategory", "Gender").sum("AmountSpent").orderBy("MemberCategory", "Gender").show()

+--------------+------+-----------------+
|MemberCategory|Gender| sum(AmountSpent)|
+--------------+------+-----------------+
|             A|     F|472.2199999999999|
|             A|     M|722.4399999999998|
|             B|     F|           216.48|
|             B|     M|           283.93|
|             C|     F|           401.51|
|             C|     M|           389.73|
+--------------+------+-----------------+



###Example 12: ROLL UP function (vs Group By)
Observe that there are more rows now. The rows with “null” represent the totals of all the grouping divisions. For instance the value reflected in B-Null is the sum of B-M and B-F

In [34]:
df_cust.rollup("MemberCategory","Gender").sum("AmountSpent") .orderBy("MemberCategory","Gender").show()

+--------------+------+------------------+
|MemberCategory|Gender|  sum(AmountSpent)|
+--------------+------+------------------+
|          NULL|  NULL| 2486.309999999999|
|             A|  NULL|           1194.66|
|             A|     F| 472.2199999999999|
|             A|     M| 722.4399999999998|
|             B|  NULL|500.40999999999997|
|             B|     F|            216.48|
|             B|     M|            283.93|
|             C|  NULL|            791.24|
|             C|     F|            401.51|
|             C|     M|            389.73|
+--------------+------+------------------+



###Example 13: The CUBE function instead of ROLL UP
Observe that while in the case of rollup the MemberCategory was taken as primary and the total for only the sub-category was performed, the cube is multidimensional. This means computation of totals is performed from both MemberCategory as primary and Gender as primary. So while rollup function provides only totals for MemberCategory (i.e., sum of all Gender for each Member Category), the cube function provides totals for all MemberCategory (i.e. sum by Gender for each Member Category) AS WELL AS the totals of all Gender (i.e., sum by all MemberCategory for each Gender).

In [35]:
df_cust.cube("MemberCategory","Gender").sum("AmountSpent").orderBy("MemberCategory","Gender").show()

+--------------+------+------------------+
|MemberCategory|Gender|  sum(AmountSpent)|
+--------------+------+------------------+
|          NULL|  NULL| 2486.309999999999|
|          NULL|     F|           1090.21|
|          NULL|     M|1396.1000000000001|
|             A|  NULL|           1194.66|
|             A|     F| 472.2199999999999|
|             A|     M| 722.4399999999998|
|             B|  NULL|500.40999999999997|
|             B|     F|            216.48|
|             B|     M|            283.93|
|             C|  NULL|            791.24|
|             C|     F|            401.51|
|             C|     M|            389.73|
+--------------+------+------------------+



###Example 14: Getting the Standard Deviation of a field
Obtaining the Standard Deviation of the Amounts that the customers spent.

In [38]:
print(df_cust.agg(stddev_pop("AmountSpent")).first())

Row(stddev_pop(AmountSpent)=28.895954726570288)


###Example 15: Getting the Skewness of a field
Obtaining the skewness of the Amounts that the customers spent.

In [40]:
print(df_cust.agg(skewness("AmountSpent")).first())

Row(skewness(AmountSpent)=-0.09241609965633672)


###Example 16: Getting the Common Statistics of a few fields in one function call
If you desire to obtain the most common statistical data of certain fields you may use the describe function as depicted below.

In [41]:
df_cust.describe("MemberCategory", "Gender", "AmountSpent", "Age").show()

+-------+--------------+------+------------------+------------------+
|summary|MemberCategory|Gender|       AmountSpent|               Age|
+-------+--------------+------+------------------+------------------+
|  count|            50|    50|                50|                50|
|   mean|          NULL|  NULL| 49.72619999999998|             41.82|
| stddev|          NULL|  NULL|29.189322194310456|13.625740945964393|
|    min|             A|     F|              2.98|                21|
|    max|             C|     M|             99.52|                64|
+-------+--------------+------+------------------+------------------+



##MULITIPLE ENTITIES
###Example 17: Joining two data sources (CSV) and retrieving data from both
We will create two dataframes to load Customers and Country data. The country table which has been derived on normalising the country specific fields has a countrycode that maps to the countrycode field of the customers data.
The following code (presented in Example 1) is augmented to include two dataframes, df1 and df2.

In [50]:
customerFilePath = "/content/drive/My Drive/data/Customer.csv"
countryFilePath = "/content/drive/My Drive/data/Country.json"
dfCustomer = ( spark.read .option("header", "true")
  .option("inferSchema", "true")
  .csv(customerFilePath) )

dfCountry = (spark.read.option("header", "true")
  .option("inferSchema", "true").json(countryFilePath))

joinDF = dfCustomer.join(dfCountry, "CountryCode")

joinDF.select("CustomerID", "CustomerName",
               "CountryCode", "CountryName", "Currency", "TimeZone").show()

+----------+--------------------+-----------+-----------+--------+--------+
|CustomerID|        CustomerName|CountryCode|CountryName|Currency|TimeZone|
+----------+--------------------+-----------+-----------+--------+--------+
|      1111|           Steven Ou|        BRA|     Brazil|     BRL|      -3|
|      1002|       Ng Choon Seng|        CAN|     Canada|     CND|      -6|
|      2828|         Cheryl Song|        FRA|     France|     FFR|       1|
|      1000|        Lou Anna Tan|        GER|    Germany|     EUR|       2|
|      4321|        Kok Kah Chee|        MAL|   Malaysia|     MYR|       8|
|      9999|Seah Yang Hwee Ra...|        SIN|  Singapore|     SGD|       8|
|      9847|       Low Soo Chiew|        SIN|  Singapore|     SGD|       8|
|      9394|      Khoo Chee Huat|        SIN|  Singapore|     SGD|       8|
|      8973|        Loke Je Hong|        SIN|  Singapore|     SGD|       8|
|      8929|        Neo Keng Hoe|        SIN|  Singapore|     SGD|       8|
|      8888|

##RETRIEVING DATA FROM OTHER DATA SOURCE TYPES
###Example 18: Retrieving Data from Json Source.

In [51]:
dfCountry.show()

+-----------+-----------+--------+--------+
|CountryCode|CountryName|Currency|TimeZone|
+-----------+-----------+--------+--------+
|        ARG|  Argentina|     ARS|      -3|
|        AUS|  Australia|     AUD|      10|
|        BEL|    Belgium|     EUR|       1|
|        BRA|     Brazil|     BRL|      -3|
|        CAN|     Canada|     CND|      -6|
|        DEN|    Denmark|     DKK|       1|
|        FIN|    Finland|     EUR|       1|
|        FRA|     France|     FFR|       1|
|        GER|    Germany|     EUR|       2|
|        IND|  Indonesia|     IDR|       7|
|        IRE|    Ireland|     EUR|      -1|
|        ITA|      Italy|     EUR|       3|
|        JAP|      Japan|     JPY|       9|
|        KOR|      Korea|     KRW|    NULL|
|        MAL|   Malaysia|     MYR|       8|
|        MEX|     Mexico|     MXV|    NULL|
|        NOR|     Norway|    NULL|    NULL|
|        POL|     Poland|    NULL|    NULL|
|        POR|   Portugal|    NULL|    NULL|
|        SIN|  Singapore|     SG

###Example 19: Retrieving Data from Json Source written in multiline format per entity.
If the json document is formatted in multiline fashion rather than single line per record as above, you would need to specify multiline option as true.

##WRITE OPERATIONS
###Example 20: Saving a CSV file
To demonstrate the write operations this simple example reads the Country.json json file and we use the df.write method to create a new CSV file.

Please note the output path is a folder (not existing) – not a json file name.
A new CSV file gets created in the specified folder with a system generated file name.


In [52]:
dfCountry.write.csv("CountryOUT")

#PRACTICE EXERCISES
Using Dataframes and SparkSQL and working on Rebu Case study files mentioned above, write Spark SQL for the following:
##A. Data retrieval using Spark SQL
1. Retrieve all Driver data (use Drivers.CSV)


2. Retrieve all Taxis and display the data in ascending order of Taxi License Plate number.


3. Retrieve all Limosine Taxies. You should display only the Taxi Number, Taxi Type, and Taxi Colour.


4. Retrieve all 4 seater Premier taxis.


##B. Aggregation and Statistical Queries (use BEAD_Rebu_TripData.CSV)
5. Determine the average distance per trip based on ALL trips in the month of January 2024.


6. Find the total fares collected grouped by Taxi Type Maxi Cab


##C. Analytics Questions
7. Determine the Average Occupancy i.e., (Number of Passengers / Passenger Capacity) for Standard Taxis.


---
END OF WORKSHOP


---


